# Monk2 Dataset
In this notebook we perform model training, selection and assessment over the Monk 2 Dataset.


In [ ]:
from exclusiveAI.components.Validation.HoldOut import parallel_hold_out, hold_out
from exclusiveAI.components.Validation.KFoldCrossValidation import validate
from exclusiveAI.components.CallBacks import EarlyStoppingCallback
from exclusiveAI.ConfiguratorGen import ConfiguratorGen
from exclusiveAI.datasets.monk import read_monk2
from exclusiveAI.utils import one_hot_encoding
from exclusiveAI.Composer import Composer
from tqdm import tqdm
import pandas as pd
import numpy as np
from exclusiveAI.utils import plot_history
import os, json

## Read Dataset

Importing training and test dataset by splitting data and test labels.

In [ ]:
training_data, training_labels, test_data, test_labels = read_monk2("../exclusiveAI/datasets/")

Encoding the training and test data using the one-hot encoding.

In [ ]:
training_data = one_hot_encoding(training_data)

In [ ]:
test_data = one_hot_encoding(test_data)

## Grid Search
After having performed a coarse-grained grid search, we perform a fine-grained grid search using the combination of best model's parameters.

In [ ]:
def read_json_files(my_dir_path):
        data = pd.DataFrame()
        for file in os.listdir(my_dir_path):
            if file.endswith('.json'):
                with open(os.path.join(my_dir_path, file), 'r') as f:
                    my_data = [data['0'] for data in json.load(f).values()][1]
                    data = pd.concat([data,  pd.DataFrame(my_data)], ignore_index=True, axis=0)
        return data
batch_size = 100
epochs = 500

final_file = "monk2_models_configs_hist3.json"

if not os.path.exists(final_file):
    dir_path = "Monk2/"
    
    all_json_data = read_json_files(dir_path)
    regularizations = all_json_data['regularization'].unique().tolist()
    learning_rates = all_json_data['learning_rate'].unique().tolist()
    momentums = all_json_data['momentum'].unique().tolist()
    num_of_layers = [1]
    num_of_units = set([unit1 if unit1!=1 else 2 for unit in all_json_data['num_of_units'] for unit1 in unit])
    initializers = all_json_data['initializers'].value_counts().index.tolist()
    activations = ['sigmoid', 'tanh']
    
    # ea = EarlyStoppingCallback(patiente_limit=50, eps=)
    # 
    myConfigurator = ConfiguratorGen(random=False, learning_rates=learning_rates, regularizations=regularizations,
                                     loss_function=['mse'], optimizer=['sgd'],
                                     activation_functions=activations,
                                     number_of_units=num_of_units, number_of_layers=num_of_layers,
                                     momentums=momentums, initializers=initializers,
                                     input_shapes=training_data.shape,
                                     verbose=False, nesterov=True,
                                     callbacks=["earlystopping"], output_activation='sigmoid', show_line=False,
                                     ).get_configs()
    len(myConfigurator)
    
    configs=[]
    if __name__ == '__main__':
        configs.append(
            parallel_hold_out(myConfigurator, training=training_data, training_target=training_labels, epochs=epochs,
                              batch_size=batch_size, num_models=100, workers=-2, number_of_initializations=2, return_models_history=True,
                              ))
    
        configs = pd.DataFrame(configs)
        # Save as json
        configs.to_json(final_file)
else: 
    with open(final_file, 'r') as f:
        configs = [data['0'] for data in json.load(f).values()]

## Hold-out
We perform model selection using the hold-out technique.

In [ ]:
my_configs = []

if __name__ == '__main__':
    my_configs.append(
        hold_out(configs[1], training=training_data, training_target=training_labels, epochs=epochs, return_models_history=True,
                    batch_size=batch_size, num_models=100, number_of_initializations=2,
                      ))

configs=my_configs[0]

## Model Assessment

In [ ]:
models = []
old_histories = configs[0]
my_configs=configs[1]
with tqdm(total=len(my_configs)) as pbar:
    for old_hist, config in zip(old_histories, my_configs):
        model = Composer(config=config).compose()
        model.train(inputs=training_data, input_label=training_labels, val=test_data, val_labels=test_labels, epochs=epochs, batch_size=batch_size, name=config['model_name'], disable_line=True)
        test_val = model.evaluate(input=test_data, input_label=test_labels)
        models.append((model.get_last()['mse'], np.std(np.array(model.history['mse'])), model.get_last()['binary_accuracy'], test_val[0], test_val[1], model.curr_epoch, old_hist['binary_accuracy'][-1],  old_hist['val_binary_accuracy'][-1], model.history['mse'],  model.history['val_mse'], model.history['mse'], model.history['binary_accuracy'], model.history['val_binary_accuracy'], Composer(config=config).compose(), config, config['num_layers'], config['num_of_units'], config['model_name']))
        pbar.update(1)

# Convert the list of tuples to a DataFrame with one column for each element in the tuple
df = pd.DataFrame(models, columns=['Score', 'History_Std', 'Accuracy', 'Test_Score', 'Test_Accuracy', 'Trained_Epochs', 'Old_Accuracy', 'Old_Accuracy_val', 'Old_History', 'Old_History_val', 'History', 'Accuracy_History', 'Val_Accuracy_History', 'Model', 'Config', 'Num_Layers', 'Num_of_Units', 'Name'])

## Filtering and Ordering Results
Sorting the DataFrame by the first element in the tuple (column 'Value')

In [ ]:
df_sorted = df.sort_values(by=['Num_Layers', 'History_Std', 'Score', 'Test_Score'])
df_sorted = df_sorted[df_sorted['Accuracy'] >= 1]
df_sorted = df_sorted[df_sorted['Old_Accuracy_val'] >= 1]
df_sorted = df_sorted[df_sorted['Old_Accuracy'] >= 1]
histories = {row[0]: row[1] for row in df_sorted[['Name', 'History']].values}
old_histories = {row[0]: row[1] for row in df_sorted[['Name', 'Old_History']].values}
old_histories_val = {row[0]: row[1] for row in df_sorted[['Name', 'Old_History_val']].values}
df_sorted

We pick the best models according to low variance.

In [ ]:
def find_least_difference_row(my_df):
    min_diff = float('inf')
    selected_row = None

    for index, row in my_df.iterrows():
        array1 = np.array(row['History'])
        # differences1 =  (np.diff(array1) - np.mean(array1)) /np.mean(array1)
        differences1 =  (np.diff(array1) / np.mean(array1))
        min_consecutive_difference = np.min(differences1)

        if min_consecutive_difference < min_diff:
            min_diff = min_consecutive_difference
            selected_row = row

    return selected_row

# Example usage:
result_row = find_least_difference_row(df_sorted)
print("Selected row:")
print(result_row)
result_row.to_csv('Monk2_train.csv')

## Plots
Plotting the final model curves.

In [ ]:
import matplotlib.pyplot as plt
def plot_history2(name, lines: dict, fig_size=(10, 6), label='mse'):
    plt.figure(figsize=fig_size)
    for elem in lines:
        plt.plot(lines[elem], label=elem)
    plt.legend(fontsize='12')
    plt.xlabel('Epochs')
    plt.ylabel(label)
    plt.savefig(name+'.eps', format='eps')
    plt.savefig(name+'.png', format='png')

# plot_history2(name='Monk1_train', lines={result_row["Name"]: result_row['History']}, fig_size=(10,8))
plot_history2(name='Monk2_KFold', lines={'HoldOut Training '+result_row["Name"]: -np.sort(-np.array(result_row['Old_History'])), 'HoldOut Test '+result_row["Name"]: -np.sort(-np.array(result_row['Old_History_val']))}, fig_size=(10,8))
plot_history2(name='Monk2_Accuracy', lines={result_row["Name"]+" Accuracy ": result_row['Accuracy_History'], result_row["Name"]+" Test Accuracy ": result_row['Val_Accuracy_History']}, fig_size=(10,8), label='Accuracy')

This is the final configuration we chose.

In [ ]:
config = {'regularization': 1e-07, 'learning_rate': 0.7, 'loss_function': 'mse', 'activation_functions': ['tanh'], 'output_activation': 'sigmoid', 'num_of_units': [4], 'num_layers': 1, 'momentum': 0.9, 'optimizer': 'sgd', 'initializers': 'uniform', 'nesterov': True, 'input_shape': [169, 17], 'callbacks': ['earlystopping_1e-4_20_False'], 'verbose': False, 'outputs': 1, 'model_name': 'Model221'}